In [4]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
%pylab --no-import-all inline

In [3]:
res = 1 # Resolution

# Square pulse
pulse = 20       # Pulse Width (ms)
period = 100     # Period (ms)

t = np.linspace(0,period,period*res)

yp = np.zeros(period*res)
yp[0:pulse*res] = 1

# Exponential decay
tau = 14        # Time decay (ms)
ye = np.exp(-t/tau)

# Convolve
y = np.convolve(yp,ye)
#y = y / max(y)     # Normalize

f, axarr = plt.subplots(3, sharex=True)
axarr[0].plot(t,yp)
axarr[0].set_title('Pulse time: ' + str(pulse) + 'ms')
axarr[0].set_ylim([0,1.1])
axarr[1].plot(t,ye)
axarr[2].plot(y[0:period])

print('Value at end of signal: {0:.2f}%'.format(y[period]/y[pulse]*100))

In [4]:
# Add noise to exponential decay
N = 100      # Number of repititions
sizN = 0.5  # Size of noise

yn = np.zeros(len(y))
for i in range(0, N):
    noise = np.random.normal(0, sizN , len(y))
    yi = y + noise
    yn+=yi
 

# Intensity can not be negative 
array_np = np.asarray(yn)
low_values_indices = array_np < 0  # Where values are low
array_np[low_values_indices] = 0  # All low values set to 0


def model_func(t, a, tau, c):
    return a*np.exp(-t / tau) + c

# Non-linear fit
guess = [1, 10, 0]      # Initial guess for fit
popt, pcov = curve_fit(model_func, t[pulse:period], yn[pulse:period], p0=guess)
a, tauP, c = popt
perr = np.sqrt(np.diag(pcov))

# fig = plt.figure()
plt.plot(t[pulse:period], yn[pulse:period], 'o',
         t[pulse:period], N*y[pulse:period], '-',
         t[pulse:period], model_func(t[pulse:period], a, tauP, c), '--')
plt.title('Lifetime is {0:.3f} +- {1:.3f} ms (1s.d.)'.format(tauP, perr[1]))

# info at: http://stackoverflow.com/questions/3938042/fitting-exponential-decay-with-no-initial-guessing